In [1]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias

In [2]:
arcpy.env.overwriteOutput = True

# TODO: Base de Datos Corporativa
bcgs_bdsde = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\1.GIS\1.2.PRO\RAISS_Lote_4\PostgreSQL-44-bcgs_geo(rodian_consulta).sde"
bd_local = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_10_TerrenosBCGS\2_2_10_TerrenosBCGS.gdb"

dict_capa_version = {'m442.210':['m442','bcgs_geo.m442.r_lc_terreno'], # ? Maria La Baja [Rural]
    'm442.215':['m442','bcgs_geo.m442.u_lc_terreno'], # ? Maria La Baja [Urbano]
    'm606.410':['m606','bcgs_geo.m606.r_lc_terreno'], # ? Repelón [Rural]
    'm606.634':['m606','bcgs_geo.m606.u_lc_terreno'] # ? Repelón [Urbano]
} 

In [3]:

arcpy.env.workspace = bcgs_bdsde

for llave, valor in dict_capa_version.items():
    for dataset in arcpy.ListDatasets():
        if dataset == 'bcgs_geo.'+valor[0]+'.Areas_Catastrales':
            for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
                if capa == valor[1]:
                    # TODO: La función de cambio de versión exige un layer_feature
                    try:
                        arcpy.MakeFeatureLayer_management(in_features=capa, out_layer='ly_'+capa)
                        # TODO: Se hace el cambio de versión
                        arcpy.management.ChangeVersion('ly_'+capa, version_type='TRANSACTIONAL', version_name=llave)

                        # TODO: Se reemplaza los puntos para que permita copiar la capa
                        
                        renombre_capa = capa.replace('.','_')

                        # TODO: Se parametriza la bd local donde se copiaran las capas asociadas a las versiones
                        ruta_destino = os.path.join(bd_local,renombre_capa)
                        # TODO: Se copian los datos
                        arcpy.CopyFeatures_management(in_features='ly_'+capa, out_feature_class=ruta_destino)

                        print(f"Capa {capa} exportada correctamente a {ruta_destino}")
                    except arcpy.ExecuteError as e:
                        print(f"Error al exportar la capa {capa}: {e}")
                    

Capa bcgs_geo.m442.r_lc_terreno exportada correctamente a C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_10_TerrenosBCGS\2_2_10_TerrenosBCGS.gdb\bcgs_geo_m442_r_lc_terreno
Capa bcgs_geo.m442.u_lc_terreno exportada correctamente a C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_10_TerrenosBCGS\2_2_10_TerrenosBCGS.gdb\bcgs_geo_m442_u_lc_terreno
Capa bcgs_geo.m606.r_lc_terreno exportada correctamente a C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_10_TerrenosBCGS\2_2_10_TerrenosBCGS.gdb\bcgs_geo_m606_r_lc_terreno
Capa bcgs_geo.m606.u_lc_terreno exportada correctamente a C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_10_TerrenosBCGS\2_2_10_TerrenosBCGS.gdb\bcgs_geo_m606_u_lc_terreno
